In [5]:
import os
from langchain_neo4j import Neo4jGraph

# os.environ["OPENAI_API_KEY"] = "sk-9f91f7f5e8eb4dfabb71c9df5f72e7d2"
# os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"
# os.environ["NEO4J_URI"] = "neo4j://localhost:7687"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] = "pixiu5331"

os.environ["NEO4J_URI"] = "neo4j+s://5f494a0d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "JPvH9AyoJVJDlbOzUml3qfyA_oWutzcMfQCJI9qRCOY"
graph = Neo4jGraph()

In [ ]:
import pandas as pd

# 读取 CSV
data = pd.read_csv("kq_knowledge_points.csv")

# 清理数据，避免 NaN
data = data.fillna("")

# 
# data = data[data['system_title'] == '专升本高数知识点']
data = data[data['system_title'] == '专升本教育理论知识点']
# data = data[data['system_title'] == '教师招聘教育学知识点']
data["id"] = range(4000, 4000 + len(data))
data

,point_id,parent_id,point_title,system_title,id
2347,341641701666197519_wx,367611902371532800_wx,语言与思维,专升本教育理论知识点,4000
2348,367611902556082176_wx,367611902543499264_wx,教育概述,专升本教育理论知识点,4001
2349,341641701666197520_wx,367611902392504320_wx,情绪情感与意志,专升本教育理论知识点,4002
2350,367611902577053696_wx,367611902543499264_wx,教育学概述,专升本教育理论知识点,4003
2351,341641701666197521_wx,367611902413475840_wx,需要与动机,专升本教育理论知识点,4004
...,...,...,...,...,...
2486,370868224416309248_wx,370869425228795904_wx,教学理论流派,专升本教育理论知识点,4139
2487,370869425228795904_wx,367611902983901184_wx,教学理论流派,专升本教育理论知识点,4140
2488,375963423433928704_wx,375963545752420352_wx,师生关系,专升本教育理论知识点,4141
2489,375963545752420352_wx,367611902870654976_wx,师生关系,专升本教育理论知识点,4142


In [15]:

delete_query = "MATCH (n) DETACH DELETE n;"
graph.query(delete_query)

for _, row in data.iterrows():
    point_id = row["point_id"]
    parent_id = row["parent_id"]
    point_title = row["point_title"]
    system_title = row["system_title"]
    
    if parent_id == "0":
        # 处理课程和一级知识点的关系
        cypher_query = f"""
        MERGE (s:课程 {{title: '{system_title}'}})
        MERGE (p:知识点 {{title: '{point_title}'}})
        set p.oid = '{point_id}'
        MERGE (s)-[:包含]->(p)
        """
    else:
        # 查找 parent_title，确保 parent_id 存在
        parent_rows = data[data["point_id"] == parent_id]

        if parent_rows.empty:
            print(f"⚠️ 警告: 未找到 parent_id={parent_id}，跳过知识点 {point_title}")
            continue  # 跳过这个知识点
        
        parent_title = parent_rows["point_title"].values[0]  # 这里不会报错了
        
        # 处理知识点之间的层级关系
        # cypher_query = f"""
        # MERGE (parent:知识点 {{title: '{parent_title}'}})
        # MERGE (child:知识点 {{title: '{point_title}'}})
        # MERGE (parent)-[:CONTAINS]->(child)
        # """
        cypher_query = f"""
        MERGE (parent:知识点 {{title: '{parent_title}'}})
        MERGE (child:知识点 {{title: '{point_title}'}})
        set child.oid = '{point_id}'
        WITH parent, child
        WHERE parent <> child
        MERGE (parent)-[:包含]->(child)
        """

    # 执行 Cypher 语句
    graph.query(cypher_query)


⚠️ 警告: 未找到 parent_id=390179232487948288_wx，跳过知识点 1


In [16]:
import re

with open("目录.txt", "r", encoding="utf-8") as file:
    text = file.read()

# 📌 正则表达式匹配 “第X章” 和 “第X节” （支持阿拉伯数字和汉字）
pattern = r"(第[一二三四五六七八九十\d]+章\s+[^第]+)|(第[一二三四五六七八九十\d]+节\s+[^第]+)"
matches = re.findall(pattern, text)

chapters = []  # 章列表 (编号, 标题)
sections = []  # 节列表 (所属章编号, 章标题, 节编号, 节标题)

previous_chapter = None
previous_section = None

for match in matches:
    chapter_match, section_match = match

    # 如果匹配到 "第X章"
    if chapter_match:
        chapter_num = re.search(r"第([一二三四五六七八九十\d]+)章", chapter_match).group(1)
        chapter_title = chapter_match.strip()
        chapters.append((chapter_num, chapter_title))

        # 存入 Neo4j
        # graph.query(f"MERGE (:章 {{title: '{chapter_title}'}})")
        graph.query(f"""
            MERGE (a:章 {{title: '{chapter_title}'}})
            MERGE (b:教材 {{title: '2023年广东省普通高校专升本考试专用教材·教育理论'}})
            MERGE (b)-[:包括]->(a)
        """)

        # 处理 "章的前后关系"
        # if previous_chapter:
        #     graph.query(f"""
        #     MATCH (a:章 {{title: '{previous_chapter}'}})
        #     MATCH (b:章 {{title: '{chapter_title}'}})
        #     MERGE (a)-[:NEXT]->(b)
        #     """)

        previous_chapter = chapter_title  # 更新上一章
        previous_section = None  # 进入新章后，清空上一节
        continue

    # 如果匹配到 "第X节"
    if section_match:
        section_num = re.search(r"第([一二三四五六七八九十\d]+)节", section_match).group(1)
        section_title = section_match.strip()
        sections.append((chapter_num, chapter_title, section_num, section_title))

        # 存入 Neo4j
        graph.query(f"MERGE (:节 {{title: '{section_title}'}})")

        # 建立 "章包含节" 关系
        graph.query(f"""
        MATCH (c:章 {{title: '{chapter_title}'}})
        MATCH (s:节 {{title: '{section_title}'}})
        MERGE (c)-[:包括]->(s)
        """)

        # 处理 "节的前后关系"
        # if previous_section:
        #     graph.query(f"""
        #     MATCH (a:节 {{title: '{previous_section}'}})
        #     MATCH (b:节 {{title: '{section_title}'}})
        #     MERGE (a)-[:NEXT]->(b)
        #     """)

        previous_section = section_title  # 更新上一节
